In [3]:
%pip install e2b_code_interpreter

  Obtaining dependency information for e2b_code_interpreter from https://files.pythonhosted.org/packages/65/6c/0eb3985185ab524be3dd8036f0e5e59155c9dd7ca4f040eadee978001f25/e2b_code_interpreter-0.0.10-py3-none-any.whl.metadata
  Obtaining dependency information for e2b>=0.17.1 from https://files.pythonhosted.org/packages/0b/6b/978f7c3d013220a4e730cddc69b5c6e6b9badede92c9c377db15480e26f5/e2b-0.17.1-py3-none-any.whl.metadata
  Obtaining dependency information for websocket-client<2.0.0,>=1.7.0 from https://files.pythonhosted.org/packages/5a/84/44687a29792a70e111c5c477230a72c4b957d88d16141199bf9acb7537a3/websocket_client-1.8.0-py3-none-any.whl.metadata
  Using cached websocket_client-1.8.0-py3-none-any.whl.metadata (8.0 kB)
  Obtaining dependency information for aenum>=3.1.11 from https://files.pythonhosted.org/packages/d0/fa/ca0c66b388624ba9dbbf35aab3a9f326bfdf5e56a7237fe8f1b600da6864/aenum-3.1.15-py3-none-any.whl.metadata
  Obtaining dependency information for jsonrpcclient>=4.0.3 from h

In [24]:
# Get your API keys or save them to .env file.
import os
from dotenv import load_dotenv
load_dotenv()

# TODO: Get your OpenAI API key
# GROQ_API_KEY = "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"
OPENAI_API_KEY = ""

# TODO: Get your E2B API key from https://e2b.dev/docs
E2B_API_KEY = ""


# # Or use 8b version
# # MODEL_NAME = "llama3-8b-8192"
# MODEL_NAME = "llama3-70b-8192"

SYSTEM_PROMPT = """you are an agricultural data analyst. you are given tasks to complete and you run python code to solve them.
- the python code runs in jupyter notebook.
- every time you call execute_python tool, the python code is executed in a separate cell. it's okay to multiple calls to execute_python.
- display visualizations using matplotlib or any other visualization library directly in the notebook. don't worry about saving the visualizations to a file.
- you have access to the internet and can make api requests.
- you also have access to the filesystem and can read/write files.
- you can install any pip package (if it exists) if you need to but the usual packages for data analysis are already preinstalled.
- you can run any python code you want, everything is running in a secure sandbox environment.

Your role is to analyze historical weather and soil data to identify patterns and trends relevant to crop production. You possess a deep understanding of how weather and soil conditions affect crop growth, providing insights to help farmers make data-driven decisions to optimize their crop production.

Your task is to interpret the data analysis results and provide agronomic recommendations. Focus on translating the identified trends into actionable advice for farmers, considering optimal planting times, irrigation strategies, fertilizer application, and crop selection based on historical data patterns. Your recommendations should be practical, sustainable, and aimed at maximizing crop yield.

The expected output is a four-paragraph report detailing agronomic recommendations based on your data analysis."""

tools = [
    {
        "type": "function",
        "function": {
          "name": "execute_python",
          "description": "Execute python code in a Jupyter notebook cell and returns any result, stdout, stderr, display_data, and error.",
          "parameters": {
              "type": "object",
              "properties": {
                  "code": {
                      "type": "string",
                      "description": "The python code to execute in a single cell."
                  }
              },
              "required": ["code"]
          }
        },
    }
]

In [26]:
def code_interpret(e2b_code_interpreter, code):
  print("Running code interpreter...")
  exec = e2b_code_interpreter.notebook.exec_cell(
    code,
    on_stderr=lambda stderr: print("[Code Interpreter]", stderr),
    on_stdout=lambda stdout: print("[Code Interpreter]", stdout),
    # You can also stream code execution results
    # on_result=...
  )

  if exec.error:
    print("[Code Interpreter ERROR]", exec.error)
  else:
    return exec.results

In [28]:
from openai import OpenAI

client = OpenAI(api_key=OPENAI_API_KEY)

def chat(e2b_code_interpreter, user_message, base64_image = None, ):
  print(f"\n{'='*50}\nUser Message: {user_message}\n{'='*50}")

  messages = [
      {
          "role": "system",
          "content": SYSTEM_PROMPT,
      },
  ]

  if base64_image is not None:
    messages.append(
        {
          "role": "user",
          "content": [
            {
              "type": "text",
              "text": user_message,
            },
            {
              "type": "image_url",
              "image_url": {
                "url": f"data:image/jpeg;base64,{base64_image}"
              }
            }
          ]
        }
    )
  else:
    messages.append(
        {"role": "user", "content": user_message},
    )

  response = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools,
    tool_choice="auto"
  )
  for choice in response.choices:
    if choice.message.tool_calls and len(choice.message.tool_calls) > 0:
      for tool_call in choice.message.tool_calls:
        if tool_call.function.name == "execute_python":
          if "code" in tool_call.function.arguments:
            code = tool_call.function.arguments["code"]
          else:
            code = tool_call.function.arguments
          print("CODE TO RUN")
          print(code)
          code_interpreter_results = code_interpret(e2b_code_interpreter, code)
          return code_interpreter_results
    else:
      print("Answer:", choice.message.content)

In [30]:

def upload_dataset(code_interpreter):
  print("Uploading dataset to Code Interpreter sandbox...")
  with open("./data.csv", "rb") as f:
    remote_path = code_interpreter.upload_file(f)
  print("Uploaded at", remote_path)

In [32]:
from e2b_code_interpreter import CodeInterpreter



with CodeInterpreter(api_key=E2B_API_KEY) as code_interpreter:
    # Upload the dataset to the code interpreter sandbox
  upload_dataset(code_interpreter)
  code_results = chat(
    code_interpreter,
   """analyze historical weather and soil data to identify patterns and trends relevant to crop production. You possess a deep understanding of how weather and soil conditions affect crop growth, providing insights to help farmers make data-driven decisions to optimize their crop production.

Your task is to interpret the data analysis results and provide agronomic recommendations. Focus on translating the identified trends into actionable advice for farmers, considering optimal planting times, irrigation strategies, fertilizer application, and crop selection based on historical data patterns. Your recommendations should be practical, sustainable, and aimed at maximizing crop yield.
 I HAVE UPLOAED THE DATA named data.csv"""
  )
  if code_results:
    first_result = code_results
  else:
    print("No code results")
    exit(0)


# This will render the image
# You can also access the data directly
# first_result.png
# first_result.jpg
# first_result.pdf
# ...
first_result

Uploading dataset to Code Interpreter sandbox...
Uploaded at /home/user/data.csv

User Message: analyze historical weather and soil data to identify patterns and trends relevant to crop production. You possess a deep understanding of how weather and soil conditions affect crop growth, providing insights to help farmers make data-driven decisions to optimize their crop production.

Your task is to interpret the data analysis results and provide agronomic recommendations. Focus on translating the identified trends into actionable advice for farmers, considering optimal planting times, irrigation strategies, fertilizer application, and crop selection based on historical data patterns. Your recommendations should be practical, sustainable, and aimed at maximizing crop yield.
 I HAVE UPLOAED THE DATA named data.csv
CODE TO RUN
import pandas as pd

# Load the data
data = pd.read_csv("data.csv")

# Display the first few rows of the dataset to understand its structure
data.head()
Running code 

[Result(                  time  temperature_2m  dew_point_2m  apparent_temperature  \
 0  2022-01-01T00:00:00             8.3           8.0                   7.1   
 1  2022-01-01T01:00:00             9.1           8.4                   8.0   
 2  2022-01-01T02:00:00             8.0           7.7                   6.7   
 3  2022-01-01T03:00:00             8.0           7.9                   6.9   
 4  2022-01-01T04:00:00             7.6           7.6                   6.4   
 
    precipitation_probability  precipitation  rain  showers  snowfall  \
 0                        0.0            0.0   0.0      0.0       0.0   
 1                        0.0            0.0   0.0      0.0       0.0   
 2                        0.0            0.0   0.0      0.0       0.0   
 3                        0.0            0.0   0.0      0.0       0.0   
 4                        0.0            0.0   0.0      0.0       0.0   
 
    snow_depth  wind_speed_10m  soil_temperature_0_to_7cm  \
 0         0.0  